# Inventory Recommendation Logic

## Objective
- Convert demand forecasts into inventory order recommendations
- Incorporate demand risk and service level
- Produce business-ready decision outputs


In [1]:
import pandas as pd
import numpy as np
import joblib


In [2]:
df = pd.read_csv(
    "../data/processed/feature_engineered_data.csv",
    parse_dates=["week"]
)

model = joblib.load("../models/demand_forecast_model.joblib")

df.head()


,store_id,product_id,week,weekly_units_sold,weekly_units_ordered,avg_inventory_level,avg_price,avg_discount,holiday_promotion,lag_1_units_sold,lag_2_units_sold,lag_4_units_sold,rolling_4wk_avg,rolling_8wk_avg,rolling_4wk_std,week_over_week_change
0,S001,P0001,2022-02-14,994,847,230.857143,61.830000,7.857143,1,708.0,1032.0,1142.0,854.75,770.125,183.628928,0.403955
1,S001,P0001,2022-02-21,1222,875,275.428571,37.281429,12.857143,1,994.0,708.0,685.0,989.00,896.875,212.229436,0.229376
2,S001,P0001,2022-02-28,1259,644,348.714286,42.995714,9.285714,1,1222.0,994.0,1032.0,1045.75,966.000,253.832458,0.030278
3,S001,P0001,2022-03-07,983,990,326.142857,67.158571,11.428571,1,1259.0,1222.0,708.0,1114.50,1003.125,146.343204,-0.219222
4,S001,P0001,2022-03-14,946,704,197.857143,42.097143,14.285714,1,983.0,1259.0,994.0,1102.50,978.625,160.774169,-0.037640


In [3]:
FEATURES = [
    "lag_1_units_sold",
    "lag_2_units_sold",
    "lag_4_units_sold",
    "rolling_4wk_avg",
    "rolling_8wk_avg",
    "rolling_4wk_std",
    "avg_price",
    "avg_discount",
    "holiday_promotion"
]

df["forecast_units"] = model.predict(df[FEATURES])


In [4]:
SERVICE_LEVEL = 0.90  # 90% service level


In [5]:
df["safety_stock"] = df["rolling_4wk_std"] * 1.65


In [6]:
df["recommended_order_qty"] = (
    df["forecast_units"] + df["safety_stock"]
).round().clip(lower=0)


In [7]:
def classify_risk(volatility):
    if volatility < 50:
        return "Low"
    elif volatility < 150:
        return "Medium"
    else:
        return "High"

df["demand_risk"] = df["rolling_4wk_std"].apply(classify_risk)


In [8]:
latest_week = df["week"].max()

decision_df = df[df["week"] == latest_week][
    [
        "store_id",
        "product_id",
        "forecast_units",
        "safety_stock",
        "recommended_order_qty",
        "demand_risk"
    ]
]


In [9]:
decision_df.head()


,store_id,product_id,forecast_units,safety_stock,recommended_order_qty,demand_risk
98,S001,P0001,470.09,849.261668,1319.0,High
197,S001,P0002,532.58,696.763013,1229.0,High
296,S001,P0003,628.15,883.774231,1512.0,High
395,S001,P0004,557.23,642.331953,1200.0,High
494,S001,P0005,602.62,702.223794,1305.0,High


In [10]:
decision_df.to_csv(
    "../data/processed/inventory_recommendations.csv",
    index=False
)


## Inventory Recommendation Summary

- Forecasted demand is adjusted with a safety buffer to achieve target service levels
- Demand volatility is used to classify inventory risk
- Each recommendation provides a single actionable order quantity per product per store
- Output is designed for direct use in inventory planning decisions
